In [41]:
#importing necessary Libraries
import pandas as pd
from datetime import date,timedelta
import folium
from folium import Marker
from folium.plugins import MarkerCluster
import math
from matplotlib import pyplot as plt
import seaborn as sns

In [42]:
# Importing Population Data From 2019 us census
population = pd.read_csv("/Users/sdevkota01/Downloads/Python/GEO/US/2019_Census_US_Population_Data_By_State_Lat_Long.csv")
population.head().append(population.tail())

,STATE,POPESTIMATE2019,lat,long
0,Alabama,4903185,32.377716,-86.300568
1,Alaska,731545,58.301598,-134.420212
2,Arizona,7278717,33.448143,-112.096962
3,Arkansas,3017804,34.746613,-92.288986
4,California,39512223,38.576668,-121.493629
46,Virginia,8535519,37.538857,-77.433640
47,Washington,7614893,47.035805,-122.905014
48,West Virginia,1792147,38.336246,-81.612328
49,Wisconsin,5822434,43.074684,-89.384445
50,Wyoming,578759,41.140259,-104.820236


In [43]:
#getting the most recent Dates for filterring the vaccine data
recent = date.today() - timedelta(days = 1)
print(recent)
print("\n")

recent = date.strftime(recent,"%Y%m%d")
print(recent)
print("\n")


recent = recent[0:4] + "-" + recent[4:6] + "-" + recent[6:8]
print(recent)
print("\n")

# changin recent to few month back because today there is no any vaccination done
recent = '2021-06-02'
print(recent)

2022-01-02


20220102


2022-01-02


2021-06-02


In [44]:
# importing Vaccination data 
vaccine = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/us_state_vaccinations.csv")
vaccine.head().append(vaccine.tail())
                


,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
0,2021-01-12,Alabama,78134.0,377025.0,70861.0,0.15,1.59,7270.0,1.45,7.69,NaN,NaN,NaN,0.207
1,2021-01-13,Alabama,84040.0,378975.0,74792.0,0.19,1.71,9245.0,1.53,7.73,5906.0,5906.0,1205.0,0.222
2,2021-01-14,Alabama,92300.0,435350.0,80480.0,NaN,1.88,NaN,1.64,8.88,8260.0,7083.0,1445.0,0.212
3,2021-01-15,Alabama,100567.0,444650.0,86956.0,0.28,2.05,13488.0,1.77,9.07,8267.0,7478.0,1525.0,0.226
4,2021-01-16,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7498.0,1529.0,NaN
22879,2021-12-26,Wyoming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1202.0,2077.0,NaN
22880,2021-12-27,Wyoming,679449.0,846065.0,321856.0,47.33,117.40,273926.0,55.61,146.19,NaN,1270.0,2194.0,0.803
22881,2021-12-28,Wyoming,681822.0,846065.0,322399.0,47.41,117.81,274405.0,55.71,146.19,2373.0,1209.0,2089.0,0.806
22882,2021-12-29,Wyoming,683069.0,846565.0,322765.0,47.46,118.02,274666.0,55.77,146.27,1247.0,1129.0,1951.0,0.807
22883,2021-12-30,Wyoming,684672.0,845705.0,323245.0,47.52,118.30,275046.0,55.85,146.12,1603.0,1035.0,1788.0,0.810


In [45]:
#filtering the recent data in accordance of date 
# here in recent date i am not getting any value thus, i am using date from the past which is 2021-06-02
vaccine_location = vaccine.loc[(vaccine.date == recent)][["location","people_vaccinated"]]
vaccine_location.info()
vaccine_location.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 65 entries, 141 to 22672
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   location           65 non-null     object 
 1   people_vaccinated  63 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.5+ KB


,location,people_vaccinated
141,Alabama,1768055.0
494,Alaska,338229.0
847,American Samoa,23806.0
1200,Arizona,3380842.0
1553,Arkansas,1196107.0


In [46]:
# merging the both the data
full = pd.merge(population,vaccine_location,left_on = 'STATE',right_on = 'location').drop(columns = 'location')
full.head()

,STATE,POPESTIMATE2019,lat,long,people_vaccinated
0,Alabama,4903185,32.377716,-86.300568,1768055.0
1,Alaska,731545,58.301598,-134.420212,338229.0
2,Arizona,7278717,33.448143,-112.096962,3380842.0
3,Arkansas,3017804,34.746613,-92.288986,1196107.0
4,California,39512223,38.576668,-121.493629,22526275.0


In [47]:
full ['percent_vaccinated'] = full['people_vaccinated'] / full['POPESTIMATE2019']
full.head()

,STATE,POPESTIMATE2019,lat,long,people_vaccinated,percent_vaccinated
0,Alabama,4903185,32.377716,-86.300568,1768055.0,0.360593
1,Alaska,731545,58.301598,-134.420212,338229.0,0.462349
2,Arizona,7278717,33.448143,-112.096962,3380842.0,0.464483
3,Arkansas,3017804,34.746613,-92.288986,1196107.0,0.396350
4,California,39512223,38.576668,-121.493629,22526275.0,0.570109


In [48]:
percent = full['people_vaccinated'].sum() / full['POPESTIMATE2019'].sum()
percent

0.49688666870160375

In [49]:
# plotting the information on map 
#base_map
Map = folium.Map(location = [42.32,-71.0589], zoom_start = 4)
Map

In [51]:
#adding points on the map
mc = MarkerCluster()
for idx, row in full.iterrows():
    if not math.isnan(row['long']) and not math.isnan(row['lat']):
        mc.add_child(Marker(location = [row['lat'],row['long']], tooltip = str(round(row['percent_vaccinated']*100,2))+'%'))
        
        
Map.add_child(mc)        